In [1]:
import os
from datetime import datetime
import argparse
from tabulate import tabulate
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from io import FileIO
import urllib.request
import urllib.error

target_channel = [

    ("UCFN4dYd4H_CsW64qZqgQY-A", "楊允兒UNA"), 
    ("UC_wI74GEkBf2rh1CqETj_9w", "阿酸_AcidR"), 
    ("UCwGfaaRbF-3ocI6leB7dYXw", "OX_死神烤肉"), 

]

In [2]:
SCOPES = ['https://www.googleapis.com/auth/youtubepartner']
CLIENT_SECRETS_FILE = '/Users/chouwilliam/OrbitNext/api-data-download/key/client_secret_v2.json'
# API_SERVICE_NAME = 'youtubeAnalytics'
API_SERVICE_NAME = 'youtubereporting'
API_VERSION = 'v1'

In [6]:
# func import

def get_authenticated_service():

  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()

  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

# Remove keyword arguments that are not set
def remove_empty_kwargs(**kwargs):

  good_kwargs = {}
  if kwargs is not None:
    for key, value in kwargs.iteritems():
      if value:
        good_kwargs[key] = value
  return good_kwargs

# Call the YouTube Reporting API's jobs.list method to retrieve reporting jobs.
def list_reporting_jobs(youtube_reporting, **kwargs):
  # Only include the onBehalfOfContentOwner keyword argument if the user
  # set a value for the --content_owner argument.
  kwargs = remove_empty_kwargs(**kwargs)

  # Retrieve the reporting jobs for the user (or content owner).
  results = youtube_reporting.jobs().list(**kwargs).execute()

  if 'jobs' in results and results['jobs']:
    jobs = results['jobs']
    for job in jobs:
      print ('Reporting job id: %s\n name: %s\n for reporting type: %s\n'
        % (job['id'], job['name'], job['reportTypeId']))
  else:
    print('No jobs found')
    return False

  return True

# Call the YouTube Reporting API's reports.list method to retrieve reports created by a job
# Only include the onBehalfOfContentOwner keyword argument if the user
# set a value for the --content_owner argument

def report_retrieve(youtube_reporting, **kwargs):

  kwargs = remove_empty_kwargs(**kwargs)

  # Retrieve available reports for the selected job
  results = youtube_reporting.jobs().reports().list(**kwargs).execute()

  if 'reports' in results and results['reports']:
    reports = results['reports']
    print(reports)
    for report in reports:
      print ('Report dates: %s to %s\n       download URL: %s\n'
        % (report['startTime'], report['endTime'], report['downloadUrl']))

# Call the YouTube Reporting API's media.download method to download the report
def download_report(youtube_reporting, report_url, local_file):
  request = youtube_reporting.media().download(
    resourceName=' '
  )
  request.uri = report_url
  fh = FileIO(local_file, mode='wb')
  # Stream/download the report in a single request
  downloader = MediaIoBaseDownload(fh, request, chunksize=-1)

  done = False
  while done is False:
    status, done = downloader.next_chunk()
    if status:
      print ('Download %d%%.' % int(status.progress() * 100))
  print ('Download Complete!')

# Prompt the user to select a job and return the specified ID
def get_job_id_from_user():

  job_id = input('Please enter the job id for the report retrieval: ')
  print ('You chose "%s" as the job Id for the report retrieval.' % job_id)
  return job_id

# Prompt the user to select a report URL and return the specified URL
def get_report_url_from_user():

  report_url = input('Please enter the report URL to download: ')
  print ('You chose "%s" to download.' % report_url)
  return report_url

In [7]:
youtube_reporting = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=850260467799-j521tmvg3b52fr37ej917qge6lvf4fd3.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutubepartner&state=4XrsY8SWad1u4MVsTPjPagGD7q1Pt9&prompt=consent&access_type=offline
Enter the authorization code: 4/1AdQt8qj2d2wdvSyUNZvMZvoTsAEU47EJwAgEbqpjHQhnnZR4iY1mfaEvviM


In [8]:
report = report_retrieve(youtube_reporting)

AttributeError: 'dict' object has no attribute 'iteritems'